In [ ]:
'''
Word2Vec es un modelo que se utiliza para aprender representaciones 
vectoriales de palabras. Estas representaciones pueden capturar muchas 
propiedades lingüísticas de las palabras, como su significado semántico, 
gramatical y hasta contextual.
'''

In [3]:
import string
from gensim.models import Word2Vec
import PyPDF2
import re

In [4]:
with open ('minecraft_texto2.txt', 'r', encoding='utf-8') as file:
    documento = file.read()

In [5]:
#limpio el texto:

documento_limpio = re.sub(r'\[editar\]', '', documento)
documento_limpio = re.sub(r'[«»]', "'", documento_limpio)

In [9]:
documento_limpio

'Minecraft es un videojuego de construcción de tipo \'mundo abierto\' o en inglés sandbox creado originalmente por el sueco Markus Persson (conocido comúnmente como \'Notch\'), que creo posteriormente Mojang Studios (actualmente parte de Microsoft).19\u200b20\u200b Está programado en el lenguaje de programación Java21\u200b para la versión Java Edition y posteriormente desarrollado en C++ para la versión de Bedrock Edition.22\u200b Fue lanzado el 17 de mayo de 2009, y después de numerosos cambios, su primera versión estable \'1.0\' fue publicada el 18 de noviembre de 2011.\n\n\nMarkus Persson, el creador de Minecraft.\nUn mes antes del lanzamiento de su versión completa se estrenó una versión para dispositivos móviles llamada Minecraft: Pocket Edition en Android,23\u200b y el 17 de noviembre del mismo año fue lanzada la misma versión para iOS, aunque posteriormente esta pasaría a ser Minecraft: Bedrock Edition.24\u200b25\u200b El 9 de mayo de 2012 fue lanzada la versión del juego para 

In [10]:
len(documento_limpio)

20838

In [6]:
#Procesamiento de datos

'''
Dividimos todo el texto frases
'''

oraciones = documento_limpio.split('.')
len(oraciones)

132

In [14]:
oraciones[0]

"Minecraft es un videojuego de construcción de tipo 'mundo abierto' o en inglés sandbox creado originalmente por el sueco Markus Persson (conocido comúnmente como 'Notch'), que creo posteriormente Mojang Studios (actualmente parte de Microsoft)"

In [7]:
#Limpiar oraciones: 
#Transformamos las oraciones en listas de palabras

oraciones_limpias = []

for oracion in oraciones:
    #Eliminar puntuación y dividir por espacios
    tokens = oracion.translate(str.maketrans('','',
                                             string.punctuation)).split()
    #Convertir a minúsculas
    tokens = [word.lower() for word in tokens if word.isalpha()]
    if tokens: #Añadimos solo si hay tokens
        oraciones_limpias.append(tokens)
    
 

In [8]:
#Entrenamiento del modelo Word2vec

model = Word2Vec(sentences=oraciones_limpias,
                 vector_size=500,
                 window=5,
                 min_count=1,
                 workers=8)


#vector_size: el tamaño del embedding

#window: cuántas palabras hacia atrás y adelante el modelo considera 
#para el contexto

#min_count: cuántas veces tiene que aparecer una palabra 
#para ser considerada dentro del modelo

#workers: cuántos procesadores deben trabajar a la vez

In [9]:
#para saber cuál es el vector de la palabra 'minecraft'
#nada, de curiosos no más
#500 dimensiones
vector = model.wv['minecraft']
vector

array([ 2.61376542e-03,  2.90901517e-03,  8.02143535e-04, -2.75423547e-04,
        5.32471808e-04, -2.76457029e-03, -3.42931133e-04,  3.84432869e-03,
        1.03751244e-03,  1.08760525e-03, -4.40577453e-04,  2.44043325e-03,
        1.72473455e-03,  2.01034080e-03,  1.98552641e-03, -2.68593035e-03,
       -1.96933094e-03, -1.03688263e-03,  5.28244302e-04,  1.43586972e-03,
        1.27738295e-03,  7.92195089e-04,  2.04267027e-03,  1.18545408e-03,
        2.70869030e-04, -1.53307209e-03, -5.61044617e-05,  2.07624654e-03,
       -3.72330123e-03, -3.66061489e-04,  1.43053115e-03, -1.41592207e-03,
        2.06768140e-03,  2.67612177e-05,  1.52996613e-03,  2.68705050e-03,
        1.17271219e-03, -1.02916232e-03, -2.41044140e-03, -2.04367989e-05,
       -2.47576297e-03,  9.43230465e-04, -1.19274680e-03,  9.97523661e-04,
       -2.97871465e-03,  4.55496192e-04, -2.02609762e-03,  2.17484846e-03,
       -8.72733828e-04, -8.12311424e-04, -2.30316567e-04, -1.92385761e-03,
       -1.77572342e-03, -

In [10]:
#Ver las palabras más cercanas: 

palabras_cercanas = model.wv.most_similar('minecraft', topn=10)
palabras_cercanas

#Cuando el conjunto de datos es muy chico, no tiene ningún sentido lo que larga

[('de', 0.6116554737091064),
 ('la', 0.5835233330726624),
 ('y', 0.5831253528594971),
 ('se', 0.5571378469467163),
 ('el', 0.5451656579971313),
 ('que', 0.5364284515380859),
 ('un', 0.5204793214797974),
 ('juego', 0.5170227289199829),
 ('del', 0.51549232006073),
 ('en', 0.5126683712005615)]

In [11]:
#para guardar
model.save("minecraft.model")
#para cargar
modelo_cargado = Word2Vec.load("minecraft.model")

In [14]:
#guardar embeddings:

model.wv.save_word2vec_format('mine_emb.txt', binary = False)
model.wv.save_word2vec_format('mine_emb.bin', binary = True)

In [16]:
#para cargar un embedding:

from gensim.models import KeyedVectors

embeddings_cargados = KeyedVectors.load_word2vec_format('mine_emb.txt', 
                                                        binary = False)

#o, si fue guardado como bin:

embeddings_cargados = KeyedVectors.load_word2vec_format('mine_emb.bin', 
                                                        binary = True)

In [17]:
def analogia(v1, v2, v3):
    similitud = embeddings_cargados.most_similar(positive=[v1, v3], negative=[v2])
    print(f"{v1} es a {v2} como {similitud[0][0]} es a {v3}")

In [18]:
analogia('minecraft','juego','java')
#lo intentó :'p

minecraft es a juego como de es a java
